In [ ]:
import sys, os, git, subprocess
from utilities import detect_lt_file, write_system_lt, delete_extra_files, \
    write_lammps_data, write_lammps_input, write_parm, rewrite_README, \
    write_lammps_mol
# detect local path
current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
path_to_molecules = git_path + '/molecules'
molecule_list = []
# detect molecules
for filename in os.listdir(path_to_molecules):
    molecule_folder = os.path.join(path_to_molecules, filename)
    if os.path.isdir(molecule_folder):
        molecule = molecule_folder[len(path_to_molecules)+1:]
        molecule_name = molecule.split("_")[0]
        molecule_formula = molecule.split("_")[1]
        print("molecule:", molecule_name, "\t", "formula:", molecule_formula)

        lt_file = detect_lt_file(molecule_folder)
        # extract the system.lt file
        write_system_lt(molecule_folder, lt_file)
        # run moltemplate
        run_moltemplate = subprocess.run(["moltemplate.sh", molecule_folder + "/system.lt"],
                                         cwd=molecule_folder, stdout=open(os.devnull, 'wb'),
                                         stderr=subprocess.DEVNULL) 
        no_charge = write_lammps_data(molecule_folder, molecule_name)

        write_lammps_mol(molecule_folder, molecule_name)

        atom_list = write_parm(molecule_folder)
        write_lammps_input(molecule_folder, no_charge, atom_list, molecule_name)
        delete_extra_files(molecule_folder)
        molecule_list.append([molecule_name, molecule_formula])
        
rewrite_README(molecule_list)